<a href="https://colab.research.google.com/github/dcolinmorgan/grph/blob/main/graphistry_X_funcoup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --extra-index-url=https://pypi.nvidia.com cuml-cu11 cudf-cu11 cugraph-cu11 pylibraft_cu11 raft_dask_cu11 dask_cudf_cu11 pylibcugraph_cu11 pylibraft_cu11


Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 GB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.5/502.5 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 GB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 520.0/520.0 MB 101.1 MB/s eta 0:00:01

In [ ]:
!pip install git+https://github.com/graphistry/pygraphistry.git@feat/gpu-featurization
!pip install -U git+https://github.com/graphistry/cu-cat.git@DT6
# !pip install -e git+https://github.com/dcolinmorgan/cu-cat.git@master#egg=cu-cat


# !git clone https://github.com/dcolinmorgan/cu-cat.git
# os.getcwd()

os.chdir('src/cu-cat')
import cu_cat
os.chdir('../..')

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
import os
from collections import Counter
import cProfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pstats import Stats
import cudf
from time import time
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_colwidth', 200)

In [ ]:
import graphistry

graphistry.register(api=3,protocol="https", server="hub.graphistry.com", username='***', password='***') ## key id, secret key
graphistry.__version__


In [1]:
species='H.sapiens'


In [ ]:
edges_df = pd.read_csv('https://funcoup.org/downloads/download.action?type=network&instanceID=24480085&fileName=FC5.0_'+species+'_full'.gz', sep='\t')


In [ ]:

edges_df.columns=edges_df.columns.str.split(':').str[1]

# nodes_df = pd.DataFrame({'n': pd.concat([edges_df['Gene1'], edges_df['Gene2']]).unique()})
# n = len(nodes_df)

# nodes_df['ind'] = nodes_df.index
# nodes_df['nc'] = nodes_df['ind'].apply(lambda v: 0x01000000 * round(255 * v / n,2))

g0 = graphistry.nodes(edges_df)\
                .bind(source="Gene1", destination="Gene2")\
                .settings(url_params={
                    'pointSize': 0.3,
                    'splashAfter': 'false',
                    'bg': '%23' + 'f0f2f6'
                })
t=time()
g1=g0.umap(feature_engine='cu_cat',engine='cuml',memoize=True)
print(time()-t)

emb2=g1._node_embedding
graphistry.nodes(emb2.reset_index(),'index').edges(g1._edges,'_src_implicit','_dst_implicit').bind(point_x="x",point_y="y").settings(url_params={"play":0}).addStyle(bg={'color': '#eee'}).plot()
# g1.plot()



# g0=g0.bind(source='source',destination='destination',edge_weight='weight')
# g0.plot()